In [1]:
import pandas as pd
import sqlite3
import os

def load_energy_data(db_path="database.db"):
    # Connect to the database
    conn = sqlite3.connect(db_path)

    try:
        # Load all tables
        devices_df = pd.read_sql_query("SELECT * FROM devices", conn)

        historical_df = pd.read_sql_query("SELECT * FROM historical_energy_readings", conn)
        historical_df["timestamp"] = pd.to_datetime(historical_df["timestamp"])

        realtime_df = pd.read_sql_query("SELECT * FROM real_time_energy_readings", conn)
        realtime_df["timestamp"] = pd.to_datetime(realtime_df["timestamp"])

        predictions_df = pd.read_sql_query("SELECT * FROM predictions", conn)
        predictions_df["timestamp"] = pd.to_datetime(predictions_df["timestamp"])

        scheduled_tasks_df = pd.read_sql_query("SELECT * FROM scheduled_tasks", conn)
        scheduled_tasks_df["scheduled_time"] = pd.to_datetime(scheduled_tasks_df["scheduled_time"], errors='coerce')

        anomalies_df = pd.read_sql_query("SELECT * FROM anomalies", conn)
        anomalies_df["timestamp"] = pd.to_datetime(anomalies_df["timestamp"])

        # Group by switch_id and count anomalies
        anomaly_counts = anomalies_df.groupby("switch_id").size().reset_index(name="anomaly_count")

    finally:
        conn.close()

    # Return all DataFrames in a dictionary
    return {
        "devices": devices_df,
        "historical": historical_df,
        "realtime": realtime_df,
        "predictions": predictions_df,
        "scheduled_tasks": scheduled_tasks_df,
        "anomalies": anomalies_df,
        "anomaly_counts": anomaly_counts
    }

In [2]:
data = load_energy_data()

data["devices"]
data["historical"]
data['realtime']
data['predictions']
data['scheduled_tasks']
# data['anomalies']
# data['anomaly_counts']

,task_id,switch_id,target_date,scheduled_time,status
0,1,wm_01,2025-04-27,NaT,not_scheduled
1,2,dw_01,2025-04-30,NaT,not_scheduled
2,3,dw_01,2025-05-10,NaT,not_scheduled


In [3]:
data["historical"].head(60)

,switch_id,timestamp,power_consumption
0,rf_01,2025-04-30 10:24:06,136.23
1,B_02,2025-04-30 10:24:06,60.99
2,B_07,2025-04-30 10:24:06,20.55
3,sp_03,2025-04-30 10:24:06,134.08
4,wm_01,2025-04-30 10:24:06,1447.81
5,dw_01,2025-04-30 10:24:06,807.96
6,B_01,2025-04-30 10:24:06,21.78
7,sp_06,2025-04-30 10:24:06,39.11
8,ac_01,2025-04-30 10:24:06,402.41
9,sp_04,2025-04-30 10:24:06,43.85


In [4]:
data["historical"]

,switch_id,timestamp,power_consumption
0,rf_01,2025-04-30 10:24:06,136.23
1,B_02,2025-04-30 10:24:06,60.99
2,B_07,2025-04-30 10:24:06,20.55
3,sp_03,2025-04-30 10:24:06,134.08
4,wm_01,2025-04-30 10:24:06,1447.81
...,...,...,...
2805,ac_01,2025-05-07 10:24:06,537.91
2806,B_09,2025-05-07 10:24:06,69.11
2807,B_07,2025-05-07 10:24:06,95.54
2808,B_08,2025-05-07 10:24:06,32.58


In [5]:
data['predictions'].head(60)

,timestamp,power_consumption
0,2025-05-07 22:28:08,3183.054981
1,2025-05-07 22:58:08,2784.015706
2,2025-05-07 23:28:08,2270.831463
3,2025-05-07 23:58:08,1720.593485
4,2025-05-08 00:28:08,1436.445018
5,2025-05-08 00:58:08,1031.760682
6,2025-05-08 01:28:08,770.175183
7,2025-05-08 01:58:08,660.043827
8,2025-05-08 02:28:08,681.721930
9,2025-05-08 02:58:08,795.529773
